In [ ]:
import os
import numpy as np
import nibabel as nib
import pydicom
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
def rotate_image(image, angle):
    return np.rot90(image, k=angle // 90)

def flip_image(image):
    return np.fliplr(image)

def rotate_and_flip_masks(masks, angle):
    rotated_masks = [rotate_image(mask, angle) for mask in masks]
    flipped_masks = [flip_image(mask) for mask in rotated_masks]
    return np.array(flipped_masks)

# Função para carregar imagens DICOM
def load_dicom_images(folder_path):
    images = []
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(".dcm")])
    for file in files:
        file_path = os.path.join(folder_path, file)
        dicom = pydicom.dcmread(file_path)
        image = dicom.pixel_array
        images.append(image)
    return np.array(images)

# Função para carregar máscaras NIfTI
def load_nifti_masks(folder_path):
    masks = []
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(".nii.gz")])
    for file in files:
        file_path = os.path.join(folder_path, file)
        print(f'Carregando arquivo: {file_path}')  # Log
        try:
            nifti_img = nib.load(file_path)
            mask_volume = nifti_img.get_fdata()
            if mask_volume.size == 0:
                print(f'Arquivo vazio: {file_path}')  # Log
            else:
                # Separar as fatias individuais do volume 3D
                for i in range(mask_volume.shape[-1]):
                    masks.append(mask_volume[:, :, i])
        except Exception as e:
            print(f'Erro ao carregar {file_path}: {e}')  # Log
    return np.array(masks)

def preprocess_images(images, target_size=(128, 128)):
    images = images / np.max(images)  # Normalização
    images = np.expand_dims(images, axis=-1)  # Adiciona canal de cor
    images_resized = np.array([tf.image.resize(image, target_size).numpy() for image in images])
    return images_resized

def preprocess_masks(masks, target_size=(128, 128)):
    masks = np.expand_dims(masks, axis=-1)  # Adiciona canal de cor
    masks_resized = np.array([tf.image.resize(mask, target_size).numpy() for mask in masks])
    return masks_resized

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf

def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

# Carregar o modelo salvo
model = load_model('C:/Users/nicol/OneDrive/Documentos/PDI-Final-Article/unet_model3.keras', 
                   custom_objects={'dice_loss': dice_loss},
                   compile=False)

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=dice_loss,
              metrics=['accuracy'])

# Função para carregar e pré-processar dados (sem máscaras)
def load_and_preprocess_images(image_folder_path, target_size=(128, 128)):
    images = load_dicom_images(image_folder_path)
    images_preprocessed = preprocess_images(images, target_size)
    return images_preprocessed

# Carregar e pré-processar dados de exam03
image_folder_path_exam03 = 'C:/Users/nicol/OneDrive/Documentos/PDI-Final-Article/dataset/exam01/data'
images_exam03 = load_and_preprocess_images(image_folder_path_exam03)

# Fazer previsões nas imagens de exam03
predictions_exam03 = model.predict(images_exam03)

import matplotlib.pyplot as plt
import math

def plot_predictions(images, predictions, num_images_per_page=5):
    num_pages = math.ceil(len(images) / num_images_per_page)
    
    for page in range(num_pages):
        start_idx = page * num_images_per_page
        end_idx = min((page + 1) * num_images_per_page, len(images))
        
        plt.figure(figsize=(20, 10))
        for i in range(start_idx, end_idx):
            plt.subplot(2, num_images_per_page, i - start_idx + 1)
            plt.imshow(images[i].squeeze(), cmap='gray')
            plt.title('Original Image')
            plt.axis('off')

            plt.subplot(2, num_images_per_page, i - start_idx + 1 + num_images_per_page)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title('Predicted Mask')
            plt.axis('off')
        
        plt.show()

# Plotar as previsões para as primeiras 5 imagens
plot_predictions(images_exam03, predictions_exam03, num_images_per_page=5)

In [ ]:
import cv2
import numpy as np

def post_process_masks(predictions, threshold=0.5, kernel_size=5, small_object_size=50, small_hole_size=50):
    processed_masks = []
    
    for mask in predictions:
        # Aplicar threshold
        _, thresholded = cv2.threshold(mask, threshold, 1, cv2.THRESH_BINARY)
        
        # Converter para uint8 para operações morfológicas
        thresholded = (thresholded * 255).astype(np.uint8)
        
        # Criar kernel para operações morfológicas
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        
        # Aplicar abertura para remover ruídos pequenos
        opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel, iterations=1)
        
        # Aplicar fechamento para remover pequenos buracos
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel, iterations=1)
        
        # Remover objetos pequenos
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(closed, connectivity=8)
        sizes = stats[1:, -1]
        nb_components = nb_components - 1
        img_without_small_objects = np.zeros((output.shape))
        for i in range(0, nb_components):
            if sizes[i] >= small_object_size:
                img_without_small_objects[output == i + 1] = 255
        
        # Preencher pequenos buracos
        img_without_small_holes = img_without_small_objects.copy()
        contours, _ = cv2.findContours(img_without_small_objects.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < small_hole_size:
                cv2.drawContours(img_without_small_holes, [cnt], 0, (255), -1)
        
        # Normalizar de volta para o intervalo [0, 1]
        processed = img_without_small_holes.astype(float) / 255.0
        
        processed_masks.append(processed)
    
    return np.array(processed_masks)


# Aplicar pós-processamento às máscaras previstas
processed_predictions = post_process_masks(predictions_exam03)

# Função para plotar as imagens originais, previsões e máscaras pós-processadas
def plot_predictions_and_processed(images, predictions, processed, num_images_per_page=5):
    num_pages = math.ceil(len(images) / num_images_per_page)
    
    for page in range():
        start_idx = page * num_images_per_page
        end_idx = min((page + 1) * num_images_per_page, len(images))
        
        plt.figure(figsize=(20, 15))
        for i in range(start_idx, end_idx):
            plt.subplot(3, num_images_per_page, i - start_idx + 1)
            plt.imshow(images[i].squeeze(), cmap='gray')
            plt.title('Original Image')
            plt.axis('off')

            plt.subplot(3, num_images_per_page, i - start_idx + 1 + num_images_per_page)
            plt.imshow(predictions[i].squeeze(), cmap='gray')
            plt.title('Predicted Mask')
            plt.axis('off')

            plt.subplot(3, num_images_per_page, i - start_idx + 1 + 2*num_images_per_page)
            plt.imshow(processed[i].squeeze(), cmap='gray')
            plt.title('Processed Mask')
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Plotar as imagens originais, previsões e máscaras pós-processadas
plot_predictions_and_processed(images_exam03, predictions_exam03, processed_predictions, num_images_per_page=5)